In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertConfig

from captum.attr import visualization as viz
from captum.attr import LayerIntegratedGradients

import re

In [2]:
# load model
device='cuda'
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base",num_labels=3)
model.to(device)
model.eval()
model.zero_grad()

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
def oh(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['

In [3]:
def process(obj):
    return re.sub(r"[!@#$'\n]", '', " ".join([str(i) for i in obj]))


In [4]:
ENTUBE = 'entube.parquet'
entube = pd.read_parquet(ENTUBE)
entube['tags'] = entube['tags'].map(process)
entube['title'] = entube['title'] + " split " + entube['tags']
entube['title'].iloc[1213]

'Lời Nguyền trên Bến Sông Full HD | Phim Việt Nam 2017 Mới Hay Nhất split lời nguyền trên bến sông phim viet nam 2017 phim việt nam 2017 phim viet nam moi nhat phim việt nam mới nhất phim việt nam phim viet nam phim việt nam hay nhất phim việt nam mới nhất 2017 phim hay 2017 phim hay phim hay chiếu rạp phim phim vn phim online phim moi 2017 phim mới lời nguyền full hd loi nguyen tren ben song Phim PhimHay'

In [5]:
entube=entube[["title","label_2"]].rename({'title': 'text', 'label_2': 'label'}, axis=1)

In [6]:
from datasets import Dataset
dataset=Dataset.from_pandas(entube)
dataset=dataset.shuffle(seed=42)
dataset[0]

{'text': '[ Bình Luận CF ] Khi thay cận chiến bằng 6 Nòng VIP - Tiền Zombie v4 split crossfire crossfire 2.0 đột kích crossfire vietnam zombie v4 hunter zombie v4 [ Bình Luận CF ] Khi thay cận chiến bằng 6 Nòng VIP - Tiền Zombie v4 Tiền Zombie v4 [ Bình Luận CF ] Khi thay cận chiến bằng 6 Nòng VIP 6 Nòng VIP - Tiền Zombie v4 thay cận chiến bằng 6 Nòng VIP cận chiến bằng 6 Nòng VIP - Tiền Zombie v4',
 'label': 2}

In [7]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets=tokenized_datasets.train_test_split(test_size=0.1,seed=123)
tokenized_datasets

Map:   0%|          | 0/26236 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 23612
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2624
    })
})

In [8]:
small_train_dataset = tokenized_datasets['train']
small_eval_dataset  = tokenized_datasets['test']

In [9]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
import evaluate
metric = evaluate.load("f1", average='micro')

In [11]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels , average='micro')

In [12]:
from transformers import TrainingArguments, Trainer


training_args = TrainingArguments(
    output_dir='test_trainer',
    do_predict= True, 
    num_train_epochs=3,#3
    evaluation_strategy="steps",
    eval_steps = 1000,
    save_steps = 1000,
    save_total_limit = 2,
    load_best_model_at_end=True,)

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = small_train_dataset,
    eval_dataset = small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [14]:
trainer.train()

C:\Users\mtthe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/8856 [00:00<?, ?it/s]

{'loss': 0.8937, 'learning_rate': 4.717705510388438e-05, 'epoch': 0.17}
{'loss': 0.7963, 'learning_rate': 4.435411020776874e-05, 'epoch': 0.34}


  0%|          | 0/328 [00:00<?, ?it/s]

{'eval_loss': 0.8159347176551819, 'eval_f1': 0.20922256097560976, 'eval_runtime': 67.028, 'eval_samples_per_second': 39.148, 'eval_steps_per_second': 4.893, 'epoch': 0.34}
{'loss': 0.7991, 'learning_rate': 4.153116531165312e-05, 'epoch': 0.51}
{'loss': 0.776, 'learning_rate': 3.870822041553749e-05, 'epoch': 0.68}


...

  0%|          | 0/328 [00:00<?, ?it/s]

{'eval_loss': 0.7958177924156189, 'eval_f1': 0.14596036585365854, 'eval_runtime': 66.1554, 'eval_samples_per_second': 39.664, 'eval_steps_per_second': 4.958, 'epoch': 2.37}
{'loss': 0.6113, 'learning_rate': 7.655826558265583e-06, 'epoch': 2.54}
{'loss': 0.5956, 'learning_rate': 4.832881662149955e-06, 'epoch': 2.71}


  0%|          | 0/328 [00:00<?, ?it/s]

{'eval_loss': 0.7647254467010498, 'eval_f1': 0.15015243902439024, 'eval_runtime': 66.0687, 'eval_samples_per_second': 39.716, 'eval_steps_per_second': 4.965, 'epoch': 2.71}
{'loss': 0.6117, 'learning_rate': 2.009936766034327e-06, 'epoch': 2.88}
{'train_runtime': 4751.2111, 'train_samples_per_second': 14.909, 'train_steps_per_second': 1.864, 'train_loss': 0.7059136177896915, 'epoch': 3.0}


TrainOutput(global_step=8856, training_loss=0.7059136177896915, metrics={'train_runtime': 4751.2111, 'train_samples_per_second': 14.909, 'train_steps_per_second': 1.864, 'train_loss': 0.7059136177896915, 'epoch': 3.0})

In [15]:
a=trainer.predict(small_eval_dataset)

  0%|          | 0/328 [00:00<?, ?it/s]

In [16]:
predict=np.argmax(a[0], axis=1)
labels=a[1]

In [17]:
from sklearn.metrics import classification_report
print(classification_report(labels, predict))

              precision    recall  f1-score   support

           0       0.68      0.79      0.73       940
           1       0.53      0.42      0.47       768
           2       0.78      0.78      0.78       916

    accuracy                           0.68      2624
   macro avg       0.66      0.66      0.66      2624
weighted avg       0.67      0.68      0.67      2624



In [18]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertConfig
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer
import torch
import matplotlib.pyplot as plt
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [20]:
# load model
model = AutoModelForSequenceClassification.from_pretrained('test_trainer/checkpoint-4000')
model.to(device)
model.eval()
model.zero_grad()

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [21]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [22]:
def predict(inputs):
    return model(inputs)[0]

In [23]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [24]:
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):

    text_ids = tokenizer.encode(text, add_special_tokens=False)
    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

def construct_input_ref_token_type_pair(input_ids, sep_ind=0):
    seq_len = input_ids.size(1)
    token_type_ids = torch.tensor([[0 if i <= sep_ind else 1 for i in range(seq_len)]], device=device)
    ref_token_type_ids = torch.zeros_like(token_type_ids, device=device)# * -1
    return token_type_ids, ref_token_type_ids

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids
    
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

In [25]:
def custom_forward(inputs):
    preds = predict(inputs)
    return torch.softmax(preds, dim = 1)[0]

In [26]:
lig = LayerIntegratedGradients(custom_forward, model.roberta.embeddings)

In [48]:
text = small_eval_dataset[0]['text']
p=text.find("split")
text=text[p+6:]
text

'Vật vờ vatvo vat vo vinh vat vo điện thoại giá rẻ video công nghệ đánh giá trên tay review smartphone giá rẻ điện thoại trung quốc điện thoại hàn quốc smartphone trung quốc giá bán giá rẻ mở hộp giá bán điện thoại huawei mate 30 pro cài google cho huawei cách cài google cho huawei cài google cho mate 30 pro huawei mate 30 pro camera huawei mate 30 pro review đánh giá huawei mate 30 pro huawei mate 30 pro vat vo huawei mate 30 pro vật vờ'

In [60]:
def explain(i,tag=False):
    p = small_eval_dataset[i]['text'].find("split")
    text = small_eval_dataset[i]['text'][p+6:] if tag else small_eval_dataset[i]['text'][:p-1]
    label = small_eval_dataset[i]['label']
    input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
    token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, sep_id)
    position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
    attention_mask = construct_attention_mask(input_ids)

    indices = input_ids[0].detach().tolist()
    all_tokens = tokenizer.convert_ids_to_tokens(indices)
    attributions, delta = lig.attribute(inputs=input_ids,
                                        baselines=ref_input_ids,
                                        n_steps=700,
                                        internal_batch_size=3,
                                        return_convergence_delta=True)
    score = predict(input_ids)
    def summarize_attributions(attributions):
        attributions = attributions.sum(dim=-1).squeeze(0)
        attributions = attributions / torch.norm(attributions)
        return attributions
    attributions_sum = np.abs(summarize_attributions(attributions).cpu())
    attributions_sum[-1]=0
    # storing couple samples in an array for visualization purposes
    score_vis = viz.VisualizationDataRecord(attributions_sum,
                                            torch.softmax(score, dim = 1)[0][torch.argmax(torch.softmax(score, dim = 1)[0])],
                                            torch.argmax(torch.softmax(score, dim = 1)[0]),
                                            label,
                                            text,
                                            attributions_sum.sum(),       
                                            all_tokens,
                                            delta)
    print('\033[1m', 'Visualization For Score', '\033[0m')
    print(viz.visualize_text([score_vis]))

In [61]:
for i in range(30):
    explain(i,tag=False)

 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
2,1 (0.49),Mate 30 Pro KHÔNG CÓ GOOGLE: đừng lo,2.61,#s Mate 30 Pro KHÔNG C@@ Ó G@@ OO@@ G@@ LE@@ : đừng lo #/s


<IPython.core.display.HTML object>
 Visualization For Score 


<IPython.core.display.HTML object>
 Visualization For Score 


<IPython.core.display.HTML object>
 Visualization For Score 


...

<IPython.core.display.HTML object>
 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (0.94),"Hài Xuân | Xuất Hành Giờ Linh | Hài Kịch Tấn Beo, Thanh Nam, Hồng Nga Hay Nhất",3.23,"#s Hài Xuân | Xuất Hành Giờ Linh | Hài Kịch Tấn Be@@ o@@ , Thanh Nam@@ , Hồng Nga Hay Nhất #/s"


<IPython.core.display.HTML object>
 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (0.70),Bác sĩ gia đình | Một ngày làm bác sĩ của Mimi | Nhạc thiếu nhi vui nhộn | BabyBus,3.38,#s Bác sĩ gia đình | Một ngày làm bác sĩ của Mim@@ i | Nhạc thiếu nhi vui nhộn | Baby@@ Bus #/s


<IPython.core.display.HTML object>


In [62]:
for i in range(30):
    explain(i,tag=True)

 Visualization For Score 


<IPython.core.display.HTML object>
 Visualization For Score 


<IPython.core.display.HTML object>
 Visualization For Score 


<IPython.core.display.HTML object>
 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
2,2 (0.52),măng khô măng rừng măng nứa măng phơi măng đi rừng sinh tồn người rừng,0.73,#s măng khô măng rừng măng nứa măng phơi măng đi rừng sinh tồn người rừng #/s


...

<IPython.core.display.HTML object>
 Visualization For Score 


<IPython.core.display.HTML object>
 Visualization For Score 


<IPython.core.display.HTML object>
 Visualization For Score 


<IPython.core.display.HTML object>
 Visualization For Score 


<IPython.core.display.HTML object>


In [64]:
def explain(i,text=""):
    #text = small_eval_dataset[i]['text']
    label = small_eval_dataset[i]['label']
    input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
    token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, sep_id)
    position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
    attention_mask = construct_attention_mask(input_ids)

    indices = input_ids[0].detach().tolist()
    all_tokens = tokenizer.convert_ids_to_tokens(indices)
    attributions, delta = lig.attribute(inputs=input_ids,
                                        baselines=ref_input_ids,
                                        n_steps=700,
                                        internal_batch_size=3,
                                        return_convergence_delta=True)
    score = predict(input_ids)
    def summarize_attributions(attributions):
        attributions = attributions.sum(dim=-1).squeeze(0)
        attributions = attributions / torch.norm(attributions)
        return attributions
    attributions_sum = np.abs(summarize_attributions(attributions).cpu())
    # storing couple samples in an array for visualization purposes
    score_vis = viz.VisualizationDataRecord(attributions_sum,
                                            torch.softmax(score, dim = 1)[0][torch.argmax(torch.softmax(score, dim = 1)[0])],
                                            torch.argmax(torch.softmax(score, dim = 1)[0]),
                                            label,
                                            text,
                                            attributions_sum.sum(),       
                                            all_tokens,
                                            delta)
    print('\033[1m', 'Visualization For Score', '\033[0m')
    print(viz.visualize_text([score_vis]))

In [30]:
explain(1,text="Tập 147 - Đám Cưới Hương Và Đăng")

 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,1 (0.53),Tập 147 - Đám Cưới Hương Và Đăng,2.20,#s Tập 147 - Đám Cưới Hương Và Đăng #/s


<IPython.core.display.HTML object>


In [31]:
explain(1,text="FAPtv Cơm Nguội: Tập 147 - Đám Cưới Hương Và Đăng")

 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,2 (0.83),FAPtv Cơm Nguội: Tập 147 - Đám Cưới Hương Và Đăng,2.16,#s F@@ AP@@ tv Cơm Ng@@ uộ@@ i@@ : Tập 147 - Đám Cưới Hương Và Đăng #/s


<IPython.core.display.HTML object>


In [32]:
explain(1,text="Xuân Hinh")

 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (0.64),Xuân Hinh,1.34,#s Xuân Hinh #/s


<IPython.core.display.HTML object>


In [33]:
explain(1,text="Vượng Râu")

 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (0.70),Vượng Râu,1.58,#s Vượng Râu #/s


<IPython.core.display.HTML object>


In [34]:
explain(1,text="Hài Học Đường")

 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,2 (0.52),Hài Học Đường,1.92,#s Hài Học Đường #/s


<IPython.core.display.HTML object>


In [35]:
explain(1,text="Hài")    

 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (0.68),Hài,1.29,#s Hài #/s


<IPython.core.display.HTML object>


In [36]:
explain(1,text="Vợ Ơi! Anh Muốn Làm Chồng - PHIM HÀI TẾT 2023")

 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (0.80),Vợ Ơi! Anh Muốn Làm Chồng - PHIM HÀI TẾT 2023,2.40,#s Vợ Ơ@@ i@@ ! Anh Muốn Làm Chồng - PH@@ IM H@@ ÀI T@@ ẾT 2023 #/s


<IPython.core.display.HTML object>


In [37]:
explain(1,text="FAPtv Cơm Nguội 285: Vợ Ơi! Anh Muốn Làm Chồng - PHIM HÀI TẾT 2023")

 Visualization For Score 


<IPython.core.display.HTML object>


In [65]:
explain(0,text="Để mị nói cho mà nghe")

 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
2,0 (0.41),Để mị nói cho mà nghe,2.03,#s Để mị nói cho mà nghe #/s


<IPython.core.display.HTML object>
